# PyMysql

### 连接数据库

In [13]:
import pymysql

db = pymysql.connect(
    	host="localhost",
    	port=3306,
    	user="root",
    	password="mysql123",
    	db="taobao",
    	charset="utf8"
)


# fetch		拿来
# fetchone(),	fetchall()
# execute	执行
# cursor	光标，游标

### 操作（增删改查）

#### 查询

In [39]:
cur = db.cursor()           # 获取游标
op = 'SELECT * FROM test LIMIT 5'
cur.execute(op)             # 执行操作


result = cur.fetchone()     # 取回执行操作的一个结果
print(result)

print("@"*75)
results = cur.fetchall()     # 取回执行操作的所有结果
for item in results:
    print(item, '\n')

('妈妈夏装时尚套装中老年人雪纺衫中年女装阔腿裤两件套40岁50衣服', 'https://g-search3.alicdn.com/img/bao/uploaded/i4/i1/TB1UpmwQFXXXXcwXVXXXXXXXXXX_!!0-item_pic.jpg', 'https://detail.tmall.com/item.htm?id=548719889438&ns=1&abbucket=0', 139.0, '18015人付款', '湖北 武汉', 'WSQ17222 ', '薇诗琪 ', '适中 ', '聚酯 ', '2017年夏季 ', '通勤 ', '条纹 ', '40-49周岁 ', '九分裤 ', '宽松 ', '两件套 ', '套头 ', '常规 ', '', '圆领 ', '常规 ', '', '', '96%及以上 ', '口袋 ')
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
('母亲节衣服夏季妈妈夏装中老年女装中年雪纺衫阔腿裤两件套装40岁', 'https://g-search3.alicdn.com/img/bao/uploaded/i4/i4/TB14i1NQVXXXXazXpXXXXXXXXXX_!!0-item_pic.jpg', 'https://detail.tmall.com/item.htm?id=549484743711&ns=1&abbucket=0', 158.0, '15721人付款', '浙江 杭州', 'P722606G0 ', '胖织缘福 ', '薄 ', '聚酯 ', '2017年夏季 ', '休闲 ', '条纹 ', '40-49周岁 ', '长裤 ', '直筒 ', '两件套 ', '套头 ', '常规 ', '', '圆领 ', '常规 ', '', '', '96%及以上 ', '') 

('中年妈妈装夏装套装中老年人女装春装上衣裤子两件套奶奶夏季短袖', 'https://g-search3.alicdn.com/img/bao/uploaded/i4/i2/TB14KeURXXXXXbAXXXXXXXXXXXX_!!0-item_pic.jpg', 'https://detail

#### 删除

In [32]:
# 删除价格低于100的商品

op = 'DELETE FROM test WHERE 价格 < 100'
cur.execute(op)

db.commit()         # 需要主动提交，以保存所至西的语句

#### 更新

In [38]:
# 把位置“江苏”“浙江”“上海”统一改为“江浙沪”

op = 'UPDATE test SET 位置 = %s WHERE 位置 IN (%s,%s,%s)'
cur.execute(op, ("江浙沪","江苏","浙江","上海"))

db.commit()

#### 插入

In [43]:
op = 'INSERT INTO test(宝贝标题, 价格, 销量, 位置) VALUES (%s,%s,%s,%s)'
cur.execute(op, ("我插入的", "298", "10000", "芜湖起飞地"))

db.commit()

### 关闭游标和数据库连接

In [44]:
cur.close()
db.close()

# MyMongoDB

### 连接数据库

#### 服务启动
`mongod --dbpath /var/lib/mongo --logpath /var/log/mongodb/mongod.log --fork`

In [84]:
import pymongo
import requests

client = pymongo.MongoClient('localhost', 27017)
hero = client["hero"]
hero_url = hero["hero_url"]

### 爬取数据，并存入数据库

In [86]:
from bs4 import BeautifulSoup
url = 'https://pvp.qq.com/web201605/herolist.shtml'
res = requests.get(url)
res.encoding='gbk'
soup = BeautifulSoup(res.text)

selector = 'body > div.wrapper > div > div > div.herolist-box > div.herolist-content > ul > li > a'
print(len(soup.select(selector=selector)))

for item in soup.select(selector=selector):
    hero_url_dict = {
        'name':item.get_text(),
        'url':'https://pvp.qq.com/web201605/'+item.get('href')
    }
    hero_url.insert(hero_url_dict)

/tmp/ipykernel_131264/1276908512.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  hero_url.insert(hero_url_dict)


> 提示

`/tmp/ipykernel_131264/1276908512.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  hero_url.insert(hero_url_dict)`

### 一些问题

> 为什么只爬了93个英雄数据呢

> 尝试将英雄的ID作为一个字段存入数据库

In [35]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
hero_db = client['hero']
hero_col = hero_db['hero_url']

hero_col.find().count()

/tmp/ipykernel_150084/1458080396.py:7: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  hero_col.find().count()


93

In [39]:
selector = 'body > div.wrapper > div > div > div.herolist-box > div.herolist-content > ul > li > a'
a = soup.select(selector=selector)
b = a[0]
b

<a href="herodetail/506.shtml" target="_blank"><img alt="云中君" height="91" src="//game.gtimg.cn/images/yxzj/img201606/heroimg/506/506.jpg" width="91"/>云中君</a>

In [50]:
selector = 'a > img'
c = b.select(selector=selector)
c[0]

<img alt="云中君" height="91" src="//game.gtimg.cn/images/yxzj/img201606/heroimg/506/506.jpg" width="91"/>

In [55]:
c[0].get('src')

'//game.gtimg.cn/images/yxzj/img201606/heroimg/506/506.jpg'

In [53]:
b.get('target')

'_blank'

In [45]:
int(a[0].get('href').split('/')[-1].split('.')[0])

506

In [54]:
b.get_text()

'云中君'

### 尝试重新存储

> lxml可以同时使用CSS选择器和Xpath选择器

英雄数量 109


In [252]:
import pymongo
import requests

client = pymongo.MongoClient('localhost', 27017)
hero_db = client['Honor_of_Kings']
hero_general_info = hero_db['hero_general_info']

if hero_general_info.find().count() == 109:
    print("跳过")
    pass
    print("再次跳过")
else:
    print("重新插入")
    hero_general_info.delete_many({})
    url = 'https://pvp.qq.com/web201605/js/herolist.json'
    res = requests.get(url)
    print("英雄数量", len(res.json()))
    for item in res.json():
        hero_types = ['战士','法师','坦克','刺客','射手','辅助']
        hero_info = {
            'ename':item['ename'],
            'cname':item['cname'],
            'type':item['hero_type'],
            'type_name':hero_types[item['hero_type']-1],
            'new_type':item['new_type'],
            'url':'https://pvp.qq.com/web201605/herodetail/' + str(item['ename']) + '.shtml'
        }
        hero_general_info.insert_one(hero_info)
print("结束")

跳过
再次跳过
结束


/tmp/ipykernel_150084/1858783224.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if hero_general_info.find().count() == 109:


In [250]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
hero_db = client['Honor_of_Kings']
hero_general_info = hero_db['hero_general_info']

hero_general_info.delete_many({})

#### xpath的一些用法

In [188]:
tree.xpath('//*[@class="herolist clearfix"]/li//text()')

['云中君',
 '瑶',
 '盘古',
 '猪八戒',
 '嫦娥',
 '上官婉儿',
 '李信',
 '沈梦溪',
 '伽罗',
 '盾山',
 '司马懿',
 '孙策',
 '元歌',
 '米莱狄',
 '狂铁',
 '弈星',
 '裴擒虎',
 '杨玉环',
 '公孙离',
 '明世隐',
 '女娲',
 '梦奇',
 '苏烈',
 '百里玄策',
 '百里守约',
 '铠',
 '鬼谷子',
 '干将莫邪',
 '东皇太一',
 '大乔',
 '黄忠',
 '诸葛亮',
 '哪吒',
 '太乙真人',
 '蔡文姬',
 '雅典娜',
 '杨戬',
 '成吉思汗',
 '钟馗',
 '虞姬',
 '李元芳',
 '张飞',
 '刘备',
 '后羿',
 '牛魔',
 '孙悟空',
 '亚瑟',
 '橘右京',
 '娜可露露',
 '不知火舞',
 '张良',
 '花木兰',
 '兰陵王',
 '王昭君',
 '韩信',
 '刘邦',
 '姜子牙',
 '露娜',
 '程咬金',
 '安琪拉',
 '貂蝉',
 '关羽',
 '老夫子',
 '武则天',
 '项羽',
 '达摩',
 '狄仁杰',
 '马可波罗',
 '李白',
 '宫本武藏',
 '典韦',
 '曹操',
 '甄姬',
 '夏侯惇',
 '周瑜',
 '吕布',
 '芈月',
 '白起',
 '扁鹊',
 '孙膑',
 '钟无艳',
 '阿轲',
 '高渐离',
 '刘禅',
 '庄周',
 '鲁班七号',
 '孙尚香',
 '嬴政',
 '妲己',
 '墨子',
 '赵云',
 '小乔',
 '廉颇']

In [183]:
tree.xpath('//*[@class="herolist clearfix"]/li/a/text()')

['云中君',
 '瑶',
 '盘古',
 '猪八戒',
 '嫦娥',
 '上官婉儿',
 '李信',
 '沈梦溪',
 '伽罗',
 '盾山',
 '司马懿',
 '孙策',
 '元歌',
 '米莱狄',
 '狂铁',
 '弈星',
 '裴擒虎',
 '杨玉环',
 '公孙离',
 '明世隐',
 '女娲',
 '梦奇',
 '苏烈',
 '百里玄策',
 '百里守约',
 '铠',
 '鬼谷子',
 '干将莫邪',
 '东皇太一',
 '大乔',
 '黄忠',
 '诸葛亮',
 '哪吒',
 '太乙真人',
 '蔡文姬',
 '雅典娜',
 '杨戬',
 '成吉思汗',
 '钟馗',
 '虞姬',
 '李元芳',
 '张飞',
 '刘备',
 '后羿',
 '牛魔',
 '孙悟空',
 '亚瑟',
 '橘右京',
 '娜可露露',
 '不知火舞',
 '张良',
 '花木兰',
 '兰陵王',
 '王昭君',
 '韩信',
 '刘邦',
 '姜子牙',
 '露娜',
 '程咬金',
 '安琪拉',
 '貂蝉',
 '关羽',
 '老夫子',
 '武则天',
 '项羽',
 '达摩',
 '狄仁杰',
 '马可波罗',
 '李白',
 '宫本武藏',
 '典韦',
 '曹操',
 '甄姬',
 '夏侯惇',
 '周瑜',
 '吕布',
 '芈月',
 '白起',
 '扁鹊',
 '孙膑',
 '钟无艳',
 '阿轲',
 '高渐离',
 '刘禅',
 '庄周',
 '鲁班七号',
 '孙尚香',
 '嬴政',
 '妲己',
 '墨子',
 '赵云',
 '小乔',
 '廉颇']

In [176]:
tree.xpath('//*[@class="herolist clearfix"]/li[1]/a//@src')

['//game.gtimg.cn/images/yxzj/img201606/heroimg/506/506.jpg']

In [177]:
tree.xpath('//*[@class="herolist clearfix"]/li[1]/a/img/@src')

['//game.gtimg.cn/images/yxzj/img201606/heroimg/506/506.jpg']

In [179]:
tree.xpath('//*[@class="herolist clearfix"]/li[1]/a/@href')

['herodetail/506.shtml']

### 查询数据

**【MongoDB 查询数据的语法格式如下】**

>`db.collection.find(query, projection)`
- `query `：可选，使用查询操作符指定查询条件
- `projection` ：可选，使用投影操作符指定返回的键。查询时返回文档中所有键值， 只需省略该参数即可（默认省略）。
如果你需要以易读的方式来读取数据，可以使用 `pretty()` 方法，语法格式如下：

> `db.col.find().pretty()`

- `pretty`：漂亮的

In [3]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
hero_db = client['hero']
hero_col = hero_db['hero_url']

for item in hero_col.find():
    print(item)

{'_id': ObjectId('626954f1962313962bfc0cf3'), 'name': '云中君', 'url': 'https://pvp.qq.com/web201605/herodetail/506.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cf4'), 'name': '瑶', 'url': 'https://pvp.qq.com/web201605/herodetail/505.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cf5'), 'name': '盘古', 'url': 'https://pvp.qq.com/web201605/herodetail/529.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cf6'), 'name': '猪八戒', 'url': 'https://pvp.qq.com/web201605/herodetail/511.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cf7'), 'name': '嫦娥', 'url': 'https://pvp.qq.com/web201605/herodetail/515.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cf8'), 'name': '上官婉儿', 'url': 'https://pvp.qq.com/web201605/herodetail/513.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cf9'), 'name': '李信', 'url': 'https://pvp.qq.com/web201605/herodetail/507.shtml'}
{'_id': ObjectId('626954f1962313962bfc0cfa'), 'name': '沈梦溪', 'url': 'https://pvp.qq.com/web201605/herodetail/312.shtml'}
{'_id': ObjectId('626954f1962313962b